# Which csv?

## database.csv or fatalencounters??? or both??

We are going to try to see if we can use the name as a temporary key to mix the two databases

In [1]:
# Importing pandas

import pandas as pd
import datetime as dt

In [13]:
# Importing the 2 databases

wpost = pd.read_csv ('../data/database.csv')
fenc = pd.read_csv ('../data/fatalencounters.csv')

In [20]:
# Let´s have a quick look at the 2 databases.
# The Washington Post one:

wpost.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2142 entries, 0 to 2141
Data columns (total 14 columns):
id                         2142 non-null int64
name                       2123 non-null object
date                       2142 non-null object
manner_of_death            2142 non-null object
armed                      2136 non-null object
age                        2099 non-null float64
gender                     2141 non-null object
race                       2039 non-null object
city                       2142 non-null object
state                      2142 non-null object
signs_of_mental_illness    2142 non-null bool
threat_level               2142 non-null object
flee                       2106 non-null object
body_camera                2142 non-null bool
dtypes: bool(2), float64(1), int64(1), object(10)
memory usage: 205.1+ KB


In [21]:
# And now the Fatal encounters one:

fenc.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20792 entries, 0 to 20791
Data columns (total 24 columns):
Timestamp                                                                          20791 non-null float64
Subject's name                                                                     20792 non-null object
Subject's age                                                                      20207 non-null object
Subject's gender                                                                   20757 non-null object
Subject's race                                                                     20791 non-null object
URL of image of deceased                                                           7084 non-null object
Date of injury resulting in death (month/day/year)                                 20792 non-null object
Location of injury (address)                                                       20315 non-null object
Location of death (city)                           

At first glance it can appear the second one to be more in everything, more entries, more columns. The first thing is true: we have indeed more entries...but...more columns?

Name/ age / gender / race, are common to both databases ... so 4 - 4

talking about place: Wpost has city / state
                     Fenc has address/city/state/zip/county
                                                        ... so 6 - 9  (3 columns of additional info about location in fenc)
                                                        
We have date, manner (or cause) of death and mental illness in both..9 - 12

We have 4 unique columns in wpost:
Armed / Threat level / Flee / Body Camera                   ...13 - 12  ( 4 columns exclusive of wpost)

We have several unique columns in fenc, but some expendable:
URL of image of deceased / A brief description of the circumstances surrounding the death / Link to news article or photo of official document / State Data Status / Date&Description / Date (Year) / 20791 are expendable (we can keep the description for curiosity purposes)
We have 2 identifiers: de unique id and the timestamp ( we just need one)
We have 2 uniqe columns: agency and official dispotition    ...so 13 - 14  (2 uniques in fenc)


We have 5 columns unique in the fenc (3 of them about the location), and 4 unique in the wpost

In [22]:
# Comparing the 2 databases entries (just a remembrance)

wpost.shape, fenc.shape

((2142, 14), (20792, 24))

### Time frame of each database

Let´s see, more or less, wich is the difference of entries between our 2 databases considering a similar time frame.

In [23]:
fenc['Date (Year)'].min(), fenc['Date (Year)'].max()

(2000, 2100)

In [24]:
# Since we have a problem here, let´s see what is going to happen in the 2100


fenc[fenc['Date (Year)'] > 2017]

,Timestamp,Subject's name,Subject's age,Subject's gender,Subject's race,URL of image of deceased,Date of injury resulting in death (month/day/year),Location of injury (address),Location of death (city),Location of death (state),...,A brief description of the circumstances surrounding the death,Official disposition of death (justified or other),Link to news article or photo of official document,Symptoms of mental illness?,Unique identifier/submitted by,"State Data Status Prior to Jan. 1, 2013. All states complete after Jan. 1, 2013",Date&Description,20791,Unique identifier,Date (Year)
20791,NaN,Items below this row have not been fact-checked.,NaN,NaN,NaN,NaN,"December 31, 2100",Items below this line have not been fact-checked.,NaN,NaN,...,NaN,NaN,http://www.fatalencounters.org,NaN,Bill jones,X,12/31/2100: http://www.fatalencounters.org,20791.0,NaN,2100


In [25]:
# So, we have like and ending entrie that has no value to our database, le´s get rid of it

fenc = fenc.drop(20791)

In [26]:
fenc['Date (Year)'].min(), fenc['Date (Year)'].max()

(2000, 2017)

So our fatal encounters database goes from 2000 to 2017. We have solved the error of the 2100 entry dropping it.

In [27]:
# Let´s see the time frame of wpost, and the number of entries

wpost.date.min(), wpost.date.max(), wpost.id.count()

('2015-01-02', '2017-02-28', 2142)

In [28]:
# Approx, in the same time frame, in the fenc:

fenc[fenc['Date (Year)'] >= 2015].shape

(3919, 24)

We can see here that approximately, the fatal encounters db is doble the size of the washington post one. Let's try to be more accurate.

Since our column about the date of the killing in the fenc database is a string, and not a date, we cannot use it. We have to transform the data. In order to do that e are going to use the datetime library. We are going to use it to give format of a date to this column. We are going to create a new column where we are going to have this date data in a proper way.



In [30]:
# We iterate through every row, using iterrows. First we create our new column, with a string type

for i, row in fenc.iterrows():

    # we define n, as our string to be transformed for every entry
    n = row['Date of injury resulting in death (month/day/year)']

    # and here we define our new column, date, that is going to be n transformed into date. For doing that, we have to
    # describe which is the original format. In our case the date is written on a string like 'January 23, 2005'.
    # If we go to the documentation of the datetime library, we see that %B stands for the complete name of the month,
    # %d stands for the day number, then a coma, and the the year in 4 digits format.
    # The date in the end is to have just the date, without hours minutes and seconds
    fenc['date'].iloc[i] = dt.datetime.strptime(n,'%B %d, %Y')

C:\Users\ReDi\Anaconda3\lib\site-packages\pandas\core\indexing.py:141: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [33]:
fenc[(fenc['date'] >= '2015-01-02') & (fenc['date'] <= '2017-02-28')].shape

(3420, 25)

So in the end, for the same time frame the washington post dataframe is 2142 entries, the fatal encounters one is 3420. Can we try to merge them? Since the second one is 1300 entries bigger, we can asume that every name in the wpost is indeed in the fatal encounters one...but we can have typos, names that don´t match, missing names...let´s investigate.


## Merging the csv's

Just for future uses, and because my computer is very slow, let's save this as a new csv

In [38]:
fenc.to_csv('fenc.csv', index = False, encoding='utf-8')

In [14]:
import pandas as pd
import datetime as dt

wpost = pd.read_csv ('../data/database.csv')
fenc = pd.read_csv ('../data/fenc.csv')

Just to avoid problems I copy everything that has to be charged again in here, to keep on with the analysis. I have uuploaded the new database, with the date added (fenc.csv).
The first thing to do will be be stablish the fenc database in the time frame of the washington post one. So we create the timefenc

In [15]:
timefenc = fenc[(fenc['date'] >= '2015-01-02') & (fenc['date'] <= '2017-02-28')]

As we said, it's probable that all the names of the wpost are included in the fenc, but, as we are going to use the name as a key, we need the wpost to be as complete as possible. Let's recall the sql analysis for the names from a few days ago. The brief result in it is as follows:

* Jamake Cason Thomas, duplicated. The entry is exactly the same. Delete one
* Daquan Antonio Westbrook, duplicated. Googling says the correct one is id 1129. We can get rid of id 2158.
* We have 19 null names
    * 1074 John Doe
    * 1570 John Doe...age 45, latin / correct shot for shot and tased
    * 1581 Tonnia Reshelle Jarvis, 48, white
    * 1584 Kevin T. Marshall, 30, white
    * 1615  John Doe...Latin
    * 1685 John Doe
    * 1848 John Doe...Latin, 35/40 age.../armed with rocks? previously tased?
    * 2037 Andrew Depeiza, 30 , Black, unarmed, not fleeing
    * 2110 John Doe, 60s
    * 2141 Walter Ricardo Pimentel, latin...correct 19th december 2016?
    * 2154 Luis Ambrosio-Aguilar, 30, latin
    * 2168 Jose Sanchez, 30 latin
    * 2164 Orande Kandie Hayes
    * 2182 Daniel Ralph Daily, 34 , white
    * 2232 David Darnell Stroughter, 50 , Black, Westchester (hood of LA?)
    * 2278 Solomon Picart, 37, Black
    * 2319 John Doe
    * 2328 John Doe
    * 2384 William Dwayne Darby, 39, White

In [16]:
wpost[wpost['name'] == 'Jamake Cason Thomas']

,id,name,date,manner_of_death,armed,age,gender,race,city,state,signs_of_mental_illness,threat_level,flee,body_camera
2049,2304,Jamake Cason Thomas,2017-02-03,shot,gun,22.0,M,B,Rowland,NC,False,attack,Car,False
2053,2294,Jamake Cason Thomas,2017-02-03,shot,gun,22.0,M,B,Rowland,NC,False,attack,Car,False


In [19]:
# First issue solved, easy

wpost = wpost.drop(2053)

In [21]:
wpost[wpost['name'] == 'Daquan Antonio Westbrook']

,id,name,date,manner_of_death,armed,age,gender,race,city,state,signs_of_mental_illness,threat_level,flee,body_camera
976,1129,Daquan Antonio Westbrook,2015-12-24,shot,gun,18.0,M,B,Charlotte,NC,False,attack,Not fleeing,False
1934,2158,Daquan Antonio Westbrook,2016-12-24,shot,gun,18.0,M,B,Charlotte,NC,False,undetermined,Not fleeing,False


In [22]:
# If we listen to our little list, we have to drop the second one. So:

wpost = wpost.drop(1934)

For the moment, we are going to leave the unknown people, or John Does, as they are right now. Let's change only the things we know.

In [47]:
# (928) 1074 John Doe
        # nothing to do
# (1382) 1570 John Doe...age 45, latin (correct shot for shot and tased, in hold)
wpost['age'].iloc[1382] = 45
wpost[wpost['id'] == 1570]
# (1385) 1581 Tonnia Reshelle Jarvis, 48, white
# (1397) 1584 Kevin T. Marshall, 30, white
# (1419) 1615  John Doe...Latin
# (1500) 1685 John Doe
# (1646) 1848 John Doe...Latin, 35/40 age.../armed with rocks? previously tased?
# (1825) 2037 Andrew Depeiza, 30 , Black, unarmed, not fleeing
# (1894) 2110 John Doe, 60s
# (1921) 2141 Walter Ricardo Pimentel, latin...correct 19th december 2016?
# (1927) 2154 Luis Ambrosio-Aguilar, 30, latin
# (1933) 2168 Jose Sanchez, 30 latin
# (1936) 2164 Orande Kandie Hayes
# (1949) 2182 Daniel Ralph Daily, 34 , white
# (2003) 2232 David Darnell Stroughter, 50 , Black, Westchester (hood of LA?)
# (2044) 2278 Solomon Picart, 37, Black
# (2077) 2319 John Doe
# (2090) 2328 John Doe
# (2133) 2384 William Dwayne Darby, 39, White

C:\Users\Nanirulu\Anaconda3\lib\site-packages\pandas\core\indexing.py:141: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


,id,name,date,manner_of_death,armed,age,gender,race,city,state,signs_of_mental_illness,threat_level,flee,body_camera
1382,1570,NaN,2016-05-26,shot,gun,45.0,M,NaN,San Antonio,TX,False,other,Not fleeing,False
